# Requesting the tweets for the users with the fake tweets

In [10]:
# import the module 
import tweepy 
import json

# assign the values accordingly 
consumer_key = "put your consumer_key"
consumer_secret = "consumer_secret"
access_key = "access_key"
access_secret = "access_secret"


# authorization of consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 

# set access to user's access key and access secret 
auth.set_access_token(access_key, access_secret) 

# calling the api 
api = tweepy.API(auth) 


# Generating the tweets of a user_id_list

In [ ]:
# Ready
def tweets_of_user_ids(user_list_id):
    user_tweets = {}
    notvalid=[]
    for id_num in user_list_id:
        user_id = id_num
        try:
            user_tweets[id_num] = api.user_timeline(user_id)
        except:
            notvalid.append(user_id)
        
# generating the tweets from the tweets object of the user
    users_dict_tweets = {}
    for user_id, statuse in user_tweets.items():
        users_dict_tweets[user_id] = []
        for tweet in statuse:
           users_dict_tweets[user_id].append(tweet.text) 
    print(notvalid)
    return users_dict_tweets

tweets = tweets_of_user_ids(df_user_id_list)
tweets

# Data Cleaning 

In [12]:
# data cleaning 
def str_clean(tweets_list):
    tweets = []
    for text in tweets_list:
        text = text.lower()
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        text = re.sub(r'rt' , '', text)
        # tokenizing the text
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        # words = [w for w in words if not w in stop_words]
        #words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        #print(words)
        tweets.append(words)
    return tweets
        
       
    

# Extracting tweets from the users dictionary
def extract_tweet(dic):
    clean_tweets={}
    for user_id, tweets_list in dic.items():
        clean_tweets[user_id]= str_clean(tweets_list)
    return clean_tweets

# Word counter
def word_counter(dic):
    tokenized_tweets = {}
    for user_id, tweets_list in dic.items():
        lis = []
        for tweet in tweets_list:
            lis.extend(word_tokenize(tweet))
        word_count = FreqDist(lis)
        tokenized_tweets[user_id] = word_count
    return tokenized_tweets


# Word counter without the stopwords 
def word_counter_stop_w(dic):
    st_w = stopwords.words('english')
    tokenized_tweets = {}
    for user_id, tweets_list in dic.items():
        lis = []
        for tweet in tweets_list:
            lis.extend(word_tokenize(tweet))
        # remove stopwords
        lis_stop_w = [word for word in lis if word not in st_w]
        word_count = FreqDist(lis_stop_w)
        tokenized_tweets[user_id] = word_count
    return tokenized_tweets

# Post_tag the tweets
def pos_tag_tweets(dic):
    pos_tag_dic = {}
    chunk_grammar = "NP: {<JJ><NN>}"
    chunk_parser = RegexpParser(chunk_grammar) 
    for user_id, tweets in dic.items():
        tweet_list = []
        for tweet in tweets:
            tweet_tokenized = word_tokenize(tweet)
            #print(tweet_tokenized)
            tweet_pos_tag = pos_tag(tweet_tokenized)
            #print(tweet_pos_tag)
            [('Now', 'RB'), ('im', 'VBZ'), ('up', 'RP'), ('watch', 'JJ'), ('60', 'CD'), ('days', 'NNS'), ('in', 'IN')]
            tweet_parser = [tag[0] for tag in tweet_pos_tag if tag[1]=='JJ']
            #print(tweet_parser)
            #tweet_list.append(chunk_parser.parse(tweet_parser))
            tweet_list.append(tweet_parser)
        pos_tag_dic[user_id] = tweet_list
    return pos_tag_dic
            

# Convert tweets to string

In [ ]:
extr_tweets = extract_tweet(tweets) 

def tweets_to_string(dicto):
    dic={}
    for user_id, string_list in dicto.items():
        dic[user_id]= ' '.join(string_list)
    return dic
extr_tweets_strings = tweets_to_string(extr_tweets)
extr_tweets_strings